In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
def get_schedule_month(dep,arr,year,month):
    response = requests.get("https://api.ryanair.com/timetable/3/schedules/{}/{}/years/{}/months/{}?apikey=__apikey__".format(dep,arr,year,month)).json()
    return response

In [7]:
parameters = {'apikey': __apikey__}
res = requests.get("http://apigateway.ryanair.com/pub/v1/core/3/routes", 
                   params = parameters)
res = res.json()

routes_cols = []
for key, value in res[0].items():
    routes_cols.append(key)
    
import pandas as pd
# empty data frame
routes = pd.DataFrame(columns = routes_cols,
                      index = range(len(res)))
index = 0
for i in res:
    for j in routes_cols:
        routes[j]
        
for i in range(len(res)):
    for j in routes_cols:
        routes[j][i] = res[i][j]
        
routes.head()

,airportFrom,airportTo,connectingAirport,newRoute,seasonalRoute,group
0,CRL,CAG,None,False,False,CITY
1,FRA,SVQ,None,False,False,GENERIC
2,PIK,CFU,None,False,False,LEISURE
3,BDS,OPO,BGY,False,False,GENERIC
4,BOD,ORK,None,False,False,LEISURE


In [14]:
route_codes = []
for i in range(len(routes)):
    dep = routes['airportFrom'][i]
    arr = routes['airportTo'][i]
    route_codes.append(tuple([dep,arr]))

len(route_codes)

4145

In [11]:
def test_jj(route_codes,year,month):
    samp_df = pd.DataFrame(columns = ['route_dep', 'route_arr',
                                  'year', 'month',
                                  'day_date','dep_time',
                                  'arr_time'])
    ind = len(samp_df)
    for i in range(len(route_codes)):
        schedules = get_schedule_month(route_codes[i][0],route_codes[i][1], year, month)
        
        if 'days' in schedules:
            for d in schedules['days']:
                day_date = d['day']
                for j in range(len(d['flights'])):
                    dep_time = d['flights'][j]['departureTime']
                    arr_time = d['flights'][j]['arrivalTime']

                    samp_df.loc[ind,:] = [route_codes[i][0],route_codes[i][1],
                                                 year,month,day_date,dep_time,
                                                 arr_time]
                    ind = ind + 1
        
    return samp_df

In [122]:
schedule_10_1000 = test_jj(route_codes[:1000], '2018', '10')

In [123]:
schedule_10_2000 = test_jj(route_codes[1000:2000], '2018', '10')

In [124]:
schedule_10_3000 = test_jj(route_codes[2000:3000], '2018','10')

In [125]:
schedule_10_4145= test_jj(route_codes[3000:], '2018', '10') 
 

In [158]:
schedule_10 = pd.concat([schedule_10_1000,schedule_10_2000,
                       schedule_10_3000,schedule_10_4145])

In [159]:
schedule_10.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61704 entries, 0 to 16954
Data columns (total 7 columns):
route_dep    61704 non-null object
route_arr    61704 non-null object
year         61704 non-null object
month        61704 non-null object
day_date     61704 non-null object
dep_time     61704 non-null object
arr_time     61704 non-null object
dtypes: object(7)
memory usage: 3.8+ MB


In [128]:
schedule_10.to_csv("schedules_oct.csv", sep = '\t')

In [166]:
year_ryan = pd.concat([schedule_1,schedule_2,
                 schedule_3,schedule_4,schedule_5,
                 schedule_6,schedule_7,
                 schedule_8,schedule_9,
                 schedule_10])

In [130]:
year_ryan.to_csv("schedules_year.csv", sep = '\t')

In [167]:
year_ryan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 607776 entries, 0 to 16954
Data columns (total 7 columns):
route_dep    607776 non-null object
route_arr    607776 non-null object
year         607776 non-null object
month        607776 non-null object
day_date     607776 non-null object
dep_time     607776 non-null object
arr_time     607776 non-null object
dtypes: object(7)
memory usage: 37.1+ MB


In [179]:
year_ryan = year_ryan.reset_index(drop = True)

In [180]:
year_ryan[['year','month','day_date']] = year_ryan[['year','month','day_date']].apply(pd.to_numeric)

In [182]:
year_ryan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 607776 entries, 0 to 607775
Data columns (total 8 columns):
route_dep    607776 non-null object
route_arr    607776 non-null object
year         607776 non-null int64
month        607776 non-null int64
day_date     607776 non-null int64
dep_time     607776 non-null object
arr_time     607776 non-null object
date         607776 non-null object
dtypes: int64(3), object(5)
memory usage: 37.1+ MB


In [187]:
year_ryan['date'] = year_ryan.year.map(str)
year_ryan['datem'] = year_ryan.month.map(str)
year_ryan['dateday'] = year_ryan.day_date.map(str)

In [188]:
year_ryan.head()

,route_dep,route_arr,year,month,day_date,dep_time,arr_time,date,datem,dateday
0,CRL,CAG,2018,1,22,20:50,23:05,2018,1,22
1,CRL,CAG,2018,1,26,17:50,20:05,2018,1,26
2,CRL,CAG,2018,1,29,20:50,23:05,2018,1,29
3,FRA,SVQ,2018,1,20,07:30,10:25,2018,1,20
4,FRA,SVQ,2018,1,23,07:30,10:25,2018,1,23


In [189]:
year_ryan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 607776 entries, 0 to 607775
Data columns (total 10 columns):
route_dep    607776 non-null object
route_arr    607776 non-null object
year         607776 non-null int64
month        607776 non-null int64
day_date     607776 non-null int64
dep_time     607776 non-null object
arr_time     607776 non-null object
date         607776 non-null object
datem        607776 non-null object
dateday      607776 non-null object
dtypes: int64(3), object(7)
memory usage: 46.4+ MB
